### Health Insurance Lead Prediction

#### Your Client FinMan is a financial services company that provides various financial services like loan, investment funds, insurance etc. to its customers. FinMan wishes to cross-sell health insurance to the existing customers who may or may not hold insurance policies with the company. The company recommend health insurance to it's customers based on their profile once these customers land on the website. Customers might browse the recommended health insurance policy and consequently fill up a form to apply. When these customers fill-up the form, their Response towards the policy is considered positive and they are classified as a lead.
####  Once these leads are acquired, the sales advisors approach them to convert and thus the company can sell proposed health insurance to these leads in a more efficient manner.
#### Now the company needs your help in building a model to predict whether the person will be interested in their proposed Health plan/policy given the information about:
#### Demographics (city, age, region etc.)
#### Information regarding holding policies of the customer
#### Recommended Policy Information

<img src="Data_Dictionary.png">

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier
from sklearn.metrics import mean_squared_error,r2_score
from sklearn.model_selection import KFold, StratifiedKFold,GroupKFold
from lightgbm import LGBMClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.neighbors import KNeighborsRegressor,KNeighborsClassifier
from catboost import CatBoostClassifier
from sklearn.metrics import mean_squared_error,r2_score,roc_auc_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import gc
from tqdm import tqdm
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', 500)

In [26]:
train_df= pd.read_csv("Data/train.csv")
test_df=pd.read_csv("Data/test.csv")
total_df=pd.concat([train_df,test_df],ignore_index=True,sort=False)

In [27]:
total_df.head()

,ID,City_Code,Region_Code,Accomodation_Type,Reco_Insurance_Type,Upper_Age,Lower_Age,Is_Spouse,Health Indicator,Holding_Policy_Duration,Holding_Policy_Type,Reco_Policy_Cat,Reco_Policy_Premium,Response
0,1,C3,3213,Rented,Individual,36,36,No,X1,14+,3.0,22,11628.0,0.0
1,2,C5,1117,Owned,Joint,75,22,No,X2,NaN,NaN,22,30510.0,0.0
2,3,C5,3732,Owned,Individual,32,32,No,NaN,1.0,1.0,19,7450.0,1.0
3,4,C24,4378,Owned,Joint,52,48,No,X1,14+,3.0,19,17780.0,0.0
4,5,C8,2190,Rented,Individual,44,44,No,X2,3.0,1.0,16,10404.0,0.0


In [28]:
total_df["Holding_Policy_Duration"].replace("14+",15,inplace=True)
total_df['Holding_Policy_Duration']=total_df['Holding_Policy_Duration'].astype(float)

In [29]:
total_df["Holding_Policy_Duration"].fillna(0,inplace=True)
total_df["Holding_Policy_Type"].fillna(0,inplace=True)
total_df["Health Indicator"].fillna("X0",inplace=True)
total_df['Age_diff'] = total_df['Upper_Age'] - total_df['Lower_Age']
total_df['Average_age'] = (total_df['Upper_Age'] + total_df['Lower_Age'])/2